# Yelp Data Challenge - Restaurant Recommender

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use("ggplot")


In [2]:
df = pd.read_csv('data/last_2_years_restaurant_reviews.csv')

In [3]:
df.head()

,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,type,useful,user_id
0,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-06-26,0,nCqdz-NW64KazpxqnDr0sQ,1,I mainly went for the ceasar salad prepared ta...,review,0,0XVzm4kVIAaH4eQAxWbhvw
1,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-06-29,0,iwx6s6yQxc7yjS7NFANZig,4,Nice atmosphere and wonderful service. I had t...,review,0,2aeNFntqY2QDZLADNo8iQQ
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-04-05,0,2HrBENXZTiitcCJfzkELgA,2,To be honest it really quit aweful. First the ...,review,0,WFhv5pMJRDPWSyLnKiWFXA
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2016-02-16,0,6YNPXoq41qTMZ2TEi0BYUA,2,"The food was decent, but the service was defin...",review,0,2S6gWE-K3DHNcKYYSgN7xA
4,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2016-02-08,1,4bQrVUiRZ642odcKCS0OhQ,2,If you're looking for craptastic service and m...,review,1,rCTVWx_Tws2jWi-K89iEyw


## 1. Clean data and get rating data 

#### Select relevant columns in the original dataframe

In [4]:
# Get business_id, user_id, stars for recommender

business_id = df['business_id'].values
user_id = df['user_id'].values
stars = df['stars'].values

#### There are many users that haven't given many reviews, exclude these users from the item-item similarity recommender


In [23]:
len(df.business_id.unique())

4353

In [11]:
df['count'] = 1
df_user_count = df.groupby('user_id').sum()


In [82]:
df_users = df_user_count[df_user_count['count']>200]
df_users

,avg_stars,cool,funny,stars,useful,count
user_id,,,,,,
B1829_hxXSEpDPEDJtYeIw,868.5,435,312,774,613,221
JaqcCU3nxReTW2cBLHounA,932.5,12,5,829,28,231
PKEzKWv_FktMm2mGPjwd0Q,824.5,2247,1611,735,3063,215
bLbSNkLggFnqwNNzzq-Ijw,2300.5,5715,4409,1931,7624,581


In [55]:
flt1 = df['user_id']==df_users.index[0]
flt2 = df['user_id']==df_users.index[1]
flt3 = df['user_id']==df_users.index[2]
flt4 = df['user_id']==df_users.index[3]

In [63]:
df_cleaned = df[flt1|flt2|flt3|flt4]


,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,type,useful,user_id,count
385,-3zffZUHoY8bQjGfPSoBKQ,Michael Mina,"[Restaurants, Bars, Nightlife, Seafood, Americ...",4.0,9,2015-07-04,7,9RcSDjgN3ifLoSKVwErmXQ,4,I had studied the menus online and had pretty ...,review,14,PKEzKWv_FktMm2mGPjwd0Q,1
617,-BS4aZAQm9u41YnB9MUASA,Argana,"[Moroccan, Restaurants]",4.5,3,2015-02-21,4,GTsscc2tTG1YxTaJ1Iyl3g,4,I had this place bookmarked for ages and final...,review,5,bLbSNkLggFnqwNNzzq-Ijw,1
871,-C8sSrFqaCxp51pyo-fQLQ,The Rice Shop,"[Asian Fusion, Specialty Food, Comfort Food, R...",4.5,15,2015-12-02,9,16FPFJ0aN0WCOYPYm-Gp1Q,4,"Formerly a dismal-at-best fried rice counter, ...",review,15,bLbSNkLggFnqwNNzzq-Ijw,1
892,-CQokjildrY7UZezXCdEBw,Toddy Shop,"[Indian, Restaurants, American (Traditional), ...",5.0,10,2016-12-12,8,alUQs2mGsfPCzqbs3H1aBw,5,Chef Hemant Kishore is doing amazing things wi...,review,13,PKEzKWv_FktMm2mGPjwd0Q,1
894,-CQokjildrY7UZezXCdEBw,Toddy Shop,"[Indian, Restaurants, American (Traditional), ...",5.0,24,2016-11-06,15,pHfISlpZ6rwHiFK2HfUpZg,5,The Toddy Shop is the epitome of what I love a...,review,27,bLbSNkLggFnqwNNzzq-Ijw,1
994,-FcZY7a7qgxTUlTvwuyJnQ,8 Noodle Bar,"[Asian Fusion, Chinese, Restaurants, Japanese]",3.0,9,2016-08-09,4,8wu0q5bPcgORe-xCpottgA,4,"After four visits to 8 Noodle Bar, I have to w...",review,11,PKEzKWv_FktMm2mGPjwd0Q,1
1054,-FcZY7a7qgxTUlTvwuyJnQ,8 Noodle Bar,"[Asian Fusion, Chinese, Restaurants, Japanese]",3.0,7,2015-09-28,7,DazOeGA-0SwSw5wmDqbiAA,3,"If you're walking through Red Rock Casino, thi...",review,9,bLbSNkLggFnqwNNzzq-Ijw,1
1258,-IWsoxH7mLJTTpU5MmWY4w,Kinh Do,"[Restaurants, Noodles, Vietnamese]",4.0,4,2015-10-02,5,xSlPCyBTxP1hyJ3_gR21DQ,4,"Mixed feelings about this place, but they have...",review,3,bLbSNkLggFnqwNNzzq-Ijw,1
1628,-NjZ5HZApXjRJaTVZeSO_w,San Salvador Restaurant,"[American (New), Salvadoran, Restaurants, Mexi...",4.0,12,2015-12-19,8,6vx28G5Ch2fkxUvR1pkNHQ,4,The pupusas revueltas here are so freaking goo...,review,14,bLbSNkLggFnqwNNzzq-Ijw,1
1659,-OEIW0dO96-492qa_luxaw,Aces & Ales,"[Music Venues, Beer Bar, Arts & Entertainment,...",4.0,0,2016-10-22,0,giJNuwmRjJFTL5OLpFVyjQ,3,This review is for 3.5 stars. \n\nI like the o...,review,0,JaqcCU3nxReTW2cBLHounA,1


#### Create utility matrix from records

In [64]:
df_utility = pd.pivot_table(data=df_cleaned, 
                            values='stars', 
                            index='user_id', 
                            columns='business_id', 
                            fill_value=0)

In [87]:
df_utility

business_id,-3zffZUHoY8bQjGfPSoBKQ,-BS4aZAQm9u41YnB9MUASA,-C8sSrFqaCxp51pyo-fQLQ,-CQokjildrY7UZezXCdEBw,-FcZY7a7qgxTUlTvwuyJnQ,-IWsoxH7mLJTTpU5MmWY4w,-NjZ5HZApXjRJaTVZeSO_w,-OEIW0dO96-492qa_luxaw,-RJ216TTIghZshCkUlD1WQ,-U7tvCtaraTQ9b0zBhpBMA,...,zcScEL0WEdFkROcnz5379g,zdE82PiD6wquvjYLyhOJNA,zdcesxBdjljRgS0QmCgkBg,zjvnqTjBp56NhMp1GrlO5g,zmltWmTpoBt5sCU-5Kzj-Q,zp-K5s3pGTWuuaVBWo6WZA,zpoZ6WyQUYff18-z4ZU1mA,zsQk990PubOHjr1YcLkQFw,zuwba6QEBIDZT0tJZmNhdQ,zwNC-Ow4eIMan2__bS9-rg
user_id,,,,,,,,,,,,,,,,,,,,,
B1829_hxXSEpDPEDJtYeIw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,4,0,5
JaqcCU3nxReTW2cBLHounA,0,0,0,0,0,0,0,3,0,0,...,0,4,0,2,3,0,5,0,0,0
PKEzKWv_FktMm2mGPjwd0Q,4,0,0,5,4,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
bLbSNkLggFnqwNNzzq-Ijw,0,4,4,5,3,4,4,0,4,0,...,3,0,3,4,0,3,4,0,3,0


In [72]:
utility_mat = df_utility.as_matrix()

array([[0, 0, 0, ..., 4, 0, 5],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 4, 4, ..., 0, 3, 0]])

## 2. Item-Item similarity recommender

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim_mat = cosine_similarity(utility_mat.T)


In [80]:
least_to_most_sim_indexes = np.argsort(item_sim_mat, axis=1)

# Neighborhoods
neighborhood_size = 75
neighborhoods = least_to_most_sim_indexes[:, -neighborhood_size:]

In [100]:
neighborhoods

array([[240, 658, 311, ..., 157, 726,   0],
       [122, 124, 654, ..., 536, 534, 479],
       [122, 124, 654, ..., 536, 534, 479],
       ..., 
       [604, 251, 244, ..., 144, 718, 959],
       [122, 124, 654, ..., 536, 534, 479],
       [604, 251, 244, ..., 144, 718, 959]])

In [108]:
user_id = 3


In [132]:
n_users = utility_mat.shape[0]
n_items = utility_mat.shape[1]

items_rated_by_this_user = utility_mat[user_id].nonzero()[0]
# Just initializing so we have somewhere to put rating preds
out = np.zeros(n_items)
for item_to_rate in xrange(n_items):
    relevant_items = np.intersect1d(neighborhoods[item_to_rate],
                                    items_rated_by_this_user,
                                    assume_unique=True)  # assume_unique speeds up intersection op
    out[item_to_rate] = np.dot(utility_mat[user_id, relevant_items] ,\
        item_sim_mat[item_to_rate, relevant_items]) / \
        item_sim_mat[item_to_rate, relevant_items].sum()


pred_ratings = np.nan_to_num(out)
print pred_ratings

[ 0.          3.22666667  3.22666667  3.44434252  3.41026882  3.22666667
  3.22666667  0.          3.22666667  0.          3.22666667  3.22666667
  3.40184832  3.22666667  0.          3.22666667  3.22666667  0.
  3.56448108  3.22666667  3.22666667  3.58061965  3.22666667  3.54784141
  0.          0.          3.49996447  3.22666667  3.4826983   0.          0.
  3.22666667  3.22666667  3.22666667  3.22666667  0.          3.3942412   0.
  0.          0.          0.          0.          3.22666667  3.55909282
  3.57205492  3.22666667  3.56751977  3.44434252  3.22666667  0.
  3.22666667  3.22666667  0.          0.          0.          3.68833603
  3.22666667  3.37006092  3.22666667  3.22666667  0.          3.22666667
  0.          0.          3.22666667  0.          3.48230852  3.22666667
  0.          0.          3.43550351  0.          3.32459752  3.22666667
  3.22666667  3.56751977  0.          3.22666667  0.          3.22666667
  3.22666667  0.          3.22666667  3.58738174  3.2266666

/Users/zhangtianqi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in double_scalars


In [133]:
pred_ratings.shape

(960,)

In [136]:
n = 10
# Get item indexes sorted by predicted rating
item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))

# exclude the items that have been rated by user
unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                if item not in items_rated_by_this_user]

unrated_index = unrated_items_by_pred_rating[-n:]
unrated_index

[842, 573, 349, 618, 610, 637, 503, 595, 256, 128]

In [165]:
print 'Top 10 restaurants recommended for the user:\n'
for index in unrated_index:
    for name in df[df['business_id']==df_utility.columns[index]]['name']:
        print name
        break

Top 10 restaurants recommended for the user:

BARDOT Brasserie
T-Bones Chophouse
Wicked Spoon
Gordon Ramsay BurGR
STK Las Vegas
Todd English P.U.B.
Nine Fine Irishmen
Fish N Bowl
The Henry
JJANGA Japanese Restaurant


## 3. Matrix Factorization recommender

In [166]:
import graphlab

In [167]:
sf = graphlab.SFrame(df[['user_id', 'business_id', 'stars']])

This non-commercial license of GraphLab Create for academic use is assigned to tiazhang@umass.edu and will expire on July 04, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1499149138.log


In [168]:
rec = graphlab.recommender.factorization_recommender.create(
            sf,
            user_id='user_id',
            item_id='business_id',
            target='stars',
            solver='als',
            side_data_factorization=False)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 347619 observations with 160703 users and 4353 items.

Data prepared in: 0.592539s

Training factorization_recommender for recommendations.

...

| 48      | 18.66s       | 0.23938                |

| 49      | 19.01s       | 0.238831               |

| FINAL   | 19.01s       | 0.238831               |

+---------+--------------+------------------------+

Optimization Complete: Iteration limit reached.

In [169]:
one_datapoint_sf = graphlab.SFrame({'user_id': ['bLbSNkLggFnqwNNzzq-Ijw'], 'business_id': ['-3zffZUHoY8bQjGfPSoBKQ']})

In [170]:
one_datapoint_sf

business_id,user_id
-3zffZUHoY8bQjGfPSoBKQ,bLbSNkLggFnqwNNzzq-Ijw


In [171]:
print "rating:", rec.predict(one_datapoint_sf)[0]

rating: 3.0850566624


In [172]:
rec.list_fields()

['adagrad_momentum_weighting',
 'additional_iterations_if_unhealthy',
 'binary_target',
 'coefficients',
 'data_load_time',
 'init_random_sigma',
 'item_id',
 'item_side_data_column_names',
 'item_side_data_column_types',
 'linear_regularization',
 'max_iterations',
 'model_name',
 'nmf',
 'num_factors',
 'num_features',
 'num_item_side_features',
 'num_items',
 'num_observations',
 'num_tempering_iterations',
 'num_user_side_features',
 'num_users',
 'observation_data_column_names',
 'random_seed',
 'regularization',
 'regularization_type',
 'sgd_convergence_interval',
 'sgd_convergence_threshold',
 'sgd_max_trial_iterations',
 'sgd_sampling_block_size',
 'sgd_step_adjustment_interval',
 'sgd_step_size',
 'sgd_trial_sample_minimum_size',
 'sgd_trial_sample_proportion',
 'side_data_factorization',
 'solver',
 'step_size_decrease_rate',
 'target',
 'tempering_regularization_start_value',
 'track_exact_loss',
 'training_rmse',
 'training_stats',
 'training_time',
 'user_id',
 'user_side_

In [173]:
rec['coefficients'] 

{'business_id': Columns:
 	business_id	str
 	linear_terms	float
 	factors	array
 
 Rows: 4353
 
 Data:
 +------------------------+--------------+-------------------------------+
 |      business_id       | linear_terms |            factors            |
 +------------------------+--------------+-------------------------------+
 | --9e1ONYQuAa-CB_Rrw7Tw |     0.0      | [0.294831573963, -0.477617... |
 | -1vfRrlnNnNJ5boOVghMPA |     0.0      | [2.23690915108, -0.2593723... |
 | -3zffZUHoY8bQjGfPSoBKQ |     0.0      | [-0.0688893571496, -0.2286... |
 | -8R_-EkGpUhBk55K9Dd4mg |     0.0      | [0.0635256543756, 0.385396... |
 | -9YyInW1wapzdNZrhQJ9dg |     0.0      | [1.16600322723, 0.83798778... |
 | -AD5PiuJHgdUcAK-Vxao2A |     0.0      | [-0.800842523575, 0.694200... |
 | -AGdGGCeTS-njB_8GkUmjQ |     0.0      | [-0.916050374508, -1.17539... |
 | -BS4aZAQm9u41YnB9MUASA |     0.0      | [-1.11336481571, -0.672759... |
 | -Bf8BQ3yMk8U2f45r2DRKw |     0.0      | [0.41498669982, -0.5020682...

In [174]:
business = rec['coefficients']['business_id']
print len(business)
print len(business['factors'][0])
user_sf = rec['coefficients']['user_id']
print len(user_sf)
print len(user_sf['factors'][0])

4353
8
160703
8


In [175]:
sf

user_id,business_id,stars
0XVzm4kVIAaH4eQAxWbhvw,--9e1ONYQuAa-CB_Rrw7Tw,1
2aeNFntqY2QDZLADNo8iQQ,--9e1ONYQuAa-CB_Rrw7Tw,4
WFhv5pMJRDPWSyLnKiWFXA,--9e1ONYQuAa-CB_Rrw7Tw,2
2S6gWE-K3DHNcKYYSgN7xA,--9e1ONYQuAa-CB_Rrw7Tw,2
rCTVWx_Tws2jWi-K89iEyw,--9e1ONYQuAa-CB_Rrw7Tw,2
TU5j2S_Ub__ojLOpD_UepQ,--9e1ONYQuAa-CB_Rrw7Tw,5
GQWk8vgYGlN9hp0XP0V05w,--9e1ONYQuAa-CB_Rrw7Tw,5
G7ISuG8XlSd4rNsEcCG2dw,--9e1ONYQuAa-CB_Rrw7Tw,5
OC_WdUmY2fK-c1SD4JqSsw,--9e1ONYQuAa-CB_Rrw7Tw,5
ymSVFNfDzSVedxOuASOHXA,--9e1ONYQuAa-CB_Rrw7Tw,4


In [176]:
from sklearn.metrics import mean_squared_error

predictions = rec.predict(sf)
rmse = np.sqrt(mean_squared_error(sf['stars'], predictions))

print "graphlab's reported rmse:", rec['training_rmse']
print "calculated rmse:", rmse  

graphlab's reported rmse: 0.238830800565
calculated rmse: 0.238830800565


In [177]:
pd.Series(sf['stars']).describe()

count    347619.000000
mean          3.814861
std           1.403646
min           1.000000
25%           3.000000
50%           4.000000
75%           5.000000
max           5.000000
dtype: float64